In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.alert import Alert
import time
import random

In [ ]:
Email = "Email"
Password = "Email"
RETRY_TIME_L_BOUND = 20
RETRY_TIME_U_BOUND = 120
WORK_LIMIT_ITERATIONS = 30
WORK_COOLDOWN_TIME = 500

def is_date_selectable(date_element):
    return "ui-state-disabled undefined" not in date_element.get_attribute("class")

def Submit():
    try:
        wait(driver, 20).until(EC.element_to_be_clickable((By.ID, "appointments_submit"))).click()

        Model_Container = wait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "reveal")))
        Cancel_Confirm = Model_Container.find_element(By.CLASS_NAME, "text-center")
        time.sleep(0.1)
        Confirm = Cancel_Confirm.find_element(By.XPATH, " //a[contains (text(), 'Conf' ) ]")
        Confirm.click()
        return True
    
    except: 
        return False

def Calender_checking():

    calendar = wait(driver, 10).until(EC.visibility_of_element_located((By.ID, "ui-datepicker-div")))
    date_elements = calendar.find_elements(By.CSS_SELECTOR, "td:not(.ui-datepicker-other-month)")
    
    # Using counter that restricts the searching days when checking the calendar for a free day
    # Start & Stop should be defined.
    
    #Count = 0
    for date_element in date_elements:

        #Count +=1
        #if Start < Count < Stop:
            if is_date_selectable(date_element):
                    #date_text = date_element.find_element(By.CSS_SELECTOR, "span.ui-state-default").text
                    #print("Checking date:", date_text)

                    date_element.click()        
                    return True
                    break
    return False
    
Booked = False

while Booked==False:
    
    try:
        
        driver = webdriver.Chrome()
    
        driver.get("https://ais.usvisa-info.com/en-ae/niv/users/sign_in")

        Email_driver = driver.find_element(By.ID, "user_email")
        Email_driver.send_keys(Email)

        Password_driver = driver.find_element(By.ID, "user_password")
        Password_driver.send_keys(Password)

        Policy_driver = driver.find_element(By.CLASS_NAME, "icheckbox")
        Policy_driver.click()

        Sign_in = driver.find_element(By.NAME, "commit")
        Sign_in.click()

        wait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "Continue"))).click()

        span_element = driver.find_element(By.CSS_SELECTOR, "h5 span.fas.fa-calendar-minus.fa-lg.fa-fw")
        span_element.click()

        reschedule_button = driver.find_element(By.XPATH, "//a[@class='button small primary small-only-expanded' and contains(text(), 'Reschedule Appointment')]")


        for _ in range(6):  
            try:
                reschedule_button.click()
                break  
            except Exception as e:
                time.sleep(0.5)  
        else:
            print("Could not click after 3 attempts.")


        Continue = True

        Count_iterations = 0

        while Continue:
            
            try:
                Next = 3
                
                wait(driver, 10).until(EC.element_to_be_clickable((By.ID, "appointments_consulate_appointment_date"))).click()

                next_button = wait(driver, 10).until(
                            EC.element_to_be_clickable((By.XPATH, "//span[@class='ui-icon ui-icon-circle-triangle-e' and contains(text(), 'Next')]"))
                )
                
                
                # Checking variable defines the number of months that we want to search for
                for _ in range(Next):
                    
                    next_button.click()
                    if not Calender_checking():

                            next_button = wait(driver, 10).until(
                                    EC.element_to_be_clickable((By.XPATH, "//span[@class='ui-icon ui-icon-circle-triangle-e' and contains(text(), 'Next')]"))
                            )
                            #next_button.click()
                            pass

                    else:
                            wait(driver, 20).until(EC.element_to_be_clickable((By.ID, "appointments_consulate_appointment_time"))).click()
                            while True: 
                                try:
                                    select_element = wait(driver, 10).until(
                                        EC.element_to_be_clickable((By.ID, "appointments_consulate_appointment_time"))
                                    )

                                    # Wait until any non-empty option becomes visible
                                    wait(driver, 10).until(
                                        EC.presence_of_element_located((By.XPATH, "//option[@value and not(text()='')]"))
                                    )

                                    select = Select(select_element)

                                    # Get all options in the select element
                                    options = select.options

                                    # Generate a random index
                                    random_index = random.randint(1, len(options)-1)  # Avoid selecting the first empty option

                                    # Select the option at the random index
                                    select.select_by_index(random_index)

                                    selected_option = select.all_selected_options[0]
                                    selected_value = selected_option.get_attribute("value")
                                    print("Selected Value:", selected_value)
                                    break

                                except: 
                                    time.sleep(0.1)

                            if Submit():                                   
                                Booked = True
                                Continue = False
                                break

                Count_iterations+=1  

                if Count_iterations==WORK_LIMIT_ITERATIONS:
                    Continue = False
                else:
                    print(Count_iterations)

                time.sleep(random.uniform(RETRY_TIME_L_BOUND,RETRY_TIME_U_BOUND))
                driver.refresh()

            except: 
                time.sleep(5)
                driver.refresh()
                
        driver.quit()
        time.sleep(WORK_COOLDOWN_TIME)
        
    except:
        time.sleep(5)
        driver.quit()